In [1]:
import os.path

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow import keras
import scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
from scipy.stats import reciprocal
import keras_tuner as kt
import pickle

2023-06-01 12:56:17.694780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
param_distribs = {
    "model__n_hidden": [1, 2, 3, 4],
    "model__n_neurons": np.arange(1, 100),
    "model__learning_rate": [3e-4, 3e-3, 3e-2],
    "model__optimizer": ['sgd', 'nesterov', 'momentum', 'adam']
}

In [5]:
def build_model(n_hidden=1, n_neurons=100, optimizer='sgd', learning_rate=3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_neurons, activation='relu', input_shape=(8,)))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons))
    model.add(keras.layers.Dense(1))
    if optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'nesterov':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    elif optimizer == 'momentum':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [6]:
es = keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [7]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


2023-06-01 12:56:19.058968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 12:56:19.059939: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 13: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.0003, model__n_hidden=1, model__n_neurons=65, model__optimizer=sgd; total time=   8.1s
Epoch 21: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.0003, model__n_hidden=1, model__n_neurons=65, model__optimizer=sgd; total time=  10.9s
Epoch 14: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.0003, model__n_hidden=1, model__n_neurons=65, model__optimizer=sgd; total time=   7.8s
Epoch 13: early stopping
121/121 [==============================] - 0s 3ms/step
[CV] END model__learning_rate=0.0003, model__n_hidden=4, model__n_neurons=56, model__optimizer=adam; total time=  11.4s
Epoch 13: early stopping
121/121 [==============================] - 0s 3ms/step
[CV] END model__learning_rate=0.0003, model__n_hidden=4, model__n_neurons=56, model__optimizer=adam; total time=   8.0s
Epo

/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 430, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs

Epoch 11: early stopping
121/121 [==============================] - 0s 2ms/step
[CV] END model__learning_rate=0.03, model__n_hidden=3, model__n_neurons=75, model__optimizer=sgd; total time=  11.0s
Epoch 11: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.03, model__n_hidden=3, model__n_neurons=75, model__optimizer=sgd; total time=  11.0s


...

/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 430, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs

Epoch 11: early stopping
121/121 [==============================] - 0s 2ms/step
[CV] END model__learning_rate=0.03, model__n_hidden=3, model__n_neurons=95, model__optimizer=nesterov; total time=  11.0s
Epoch 11: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.03, model__n_hidden=3, model__n_neurons=95, model__optimizer=nesterov; total time=  10.9s


/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 430, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1120, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/scikeras/wrappers.py", line 1717, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs

Epoch 11: early stopping


RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7efc145b67a0>], model=<function build_model at 0x7efc145c23b0>),
                   param_distributions={'model__learning_rate': [0.0003, 0.003,
                                                                 0.03],
                                        'model__n_hidden': [1, 2, 3, 4],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

In [8]:
best_parameters = rnd_search_cv.best_params_
param_distribs["model__optimizer"] = best_parameters['model__optimizer']
param_distribs['model__n_neurons'] = best_parameters['model__n_neurons']
param_distribs['model__n_hidden'] = best_parameters['model__n_hidden']
param_distribs["model__learning_rate"] = best_parameters["model__learning_rate"]
print(param_distribs)

{'model__n_hidden': 3, 'model__n_neurons': 87, 'model__learning_rate': 0.003, 'model__optimizer': 'nesterov'}


In [9]:
with open('rnd_search_params.pkl', 'wb') as filename:
    pickle.dump(param_distribs, filename)

In [10]:
with open('rnd_search_scikeras.pkl', 'wb') as filename:
    pickle.dump(rnd_search_cv, filename)

INFO:tensorflow:Assets written to: ram://7ba39abd7d11475eb6995f07c9a8ef86/assets


INFO:tensorflow:Assets written to: ram://7ba39abd7d11475eb6995f07c9a8ef86/assets


In [11]:
def build_model_kt(hp):
    n_hidden = hp.Int('n_hidden', min_value=1, max_value=3, default=2)
    n_neurons = hp.Int('n_neurons', min_value=1, max_value=100, default=30)
    learning_rate = hp.Float('learning_rate', min_value=3e-4, max_value=3e-2, default=3e-3)
    optimizer = hp.Choice('optimizer', values=['sgd', 'nesterov', 'momentum', 'adam'], default='sgd')

    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    elif optimizer == "momentum":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    model = keras.Sequential()
    model.add(keras.layers.Dense(n_neurons, activation="relu", input_shape=(8,)))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))

    model.compile(optimizer=optimizer, loss="mse", metrics=['mse'])
    return model

In [12]:
random_search_tuner = kt.RandomSearch(build_model_kt, objective='val_mse', max_trials=10, overwrite=True, directory='my_california_housing', project_name='my_rnd_search', seed=42)
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = keras.callbacks.TensorBoard(root_logdir)

In [14]:
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb, es])

Trial 10 Complete [00h 00m 13s]
val_mse: nan

Best val_mse So Far: 0.27076995372772217
Total elapsed time: 00h 02m 11s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [16]:
best_parameters = random_search_tuner.get_best_hyperparameters(num_trials=1)[0].values
tmp_distribs = dict()
tmp_distribs['n_hidden'] = best_parameters['n_hidden']
tmp_distribs['n_neurons'] = best_parameters['n_neurons']
tmp_distribs['learning_rate'] = best_parameters['learning_rate']
tmp_distribs['optimizer'] = best_parameters['optimizer']
print(tmp_distribs)

{'n_hidden': 3, 'n_neurons': 46, 'learning_rate': 0.011305532240560318, 'optimizer': 'adam'}


In [17]:
with open('kt_search_params.pkl', 'wb') as filename:
    pickle.dump(tmp_distribs, filename)

In [18]:
best_model = random_search_tuner.get_best_models(num_models=1)[0]
best_model.save('kt_best_model.h5')